Este trabajo de ML se usará para predecir el riesgo de impago de préstamo al momento de la solicitud

In [102]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [103]:
df = pd.read_csv("Anonymize_Loan_Default_data.csv", encoding="latin1")
df = df.iloc[:, 1:]  # elimino la columna "Unnamed: 0"


In [104]:
# Vemos la estructura del dataset
df.head(15)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,repay_fail
0,2,2,0.0,0.0,0.00000,36 months,0.00,0.00,< 1 year,RENT,0.0,Not Verified,Dec-99,Charged Off,major_purchase,000xx,ZZ,0.00,0.0,Jan-01,0.0,NaN,0.0,0.0,0.0,0.00%,1.0,0.000000,0.00,0.00,0.00,Jan-07,0.00,Jan-07,Jan-07,1
1,545583,703644,2500.0,2500.0,2500.00000,36 months,13.98,85.42,4 years,RENT,20004.0,Not Verified,Jul-10,Does not meet the credit policy. Status:Fully ...,other,487xx,MI,19.86,0.0,Aug-05,5.0,NaN,7.0,0.0,981.0,21.30%,10.0,3075.291779,3075.29,2500.00,575.29,Jul-13,90.85,Aug-13,Jun-16,0
2,532101,687836,5000.0,5000.0,5000.00000,36 months,15.95,175.67,4 years,RENT,59000.0,Not Verified,Jun-10,Charged Off,debt_consolidation,115xx,NY,19.57,0.0,Apr-94,1.0,59.0,7.0,0.0,18773.0,99.90%,15.0,2948.760000,2948.76,1909.02,873.81,Nov-11,175.67,NaN,Mar-12,1
3,877788,1092507,7000.0,7000.0,7000.00000,36 months,9.91,225.58,10+ years,MORTGAGE,53796.0,Not Verified,Sep-11,Fully Paid,other,751xx,TX,10.80,3.0,Mar-98,3.0,3.0,7.0,0.0,3269.0,47.20%,20.0,8082.391880,8082.39,7000.00,1082.39,Mar-14,1550.27,NaN,Mar-14,0
4,875406,1089981,2000.0,2000.0,2000.00000,36 months,5.42,60.32,10+ years,RENT,30000.0,Not Verified,Sep-11,Fully Paid,debt_consolidation,112xx,NY,3.60,0.0,Jan-75,0.0,72.0,7.0,0.0,0.0,0%,15.0,2161.663244,2161.66,2000.00,161.66,Feb-14,53.12,NaN,Jun-16,0
5,506439,652909,3600.0,3600.0,3600.00000,36 months,10.25,116.59,10+ years,MORTGAGE,675048.0,Not Verified,Apr-10,Does not meet the credit policy. Status:Fully ...,other,352xx,AL,1.55,0.0,Apr-98,4.0,25.0,8.0,0.0,0.0,0%,25.0,4206.031191,4206.03,3600.00,606.03,May-13,146.75,Jun-13,Jun-16,0
6,981465,1204637,8000.0,8000.0,8000.00000,36 months,6.03,243.49,NaN,MORTGAGE,77736.0,Verified,Oct-11,Fully Paid,other,853xx,AZ,6.07,0.0,Jul-96,0.0,NaN,12.0,0.0,4182.0,13.60%,49.0,8724.971815,8724.97,8000.00,724.97,Apr-14,1423.66,NaN,Apr-14,0
7,749050,948200,6000.0,6000.0,6000.00000,36 months,7.49,186.61,3 years,RENT,35000.0,Not Verified,May-11,Fully Paid,debt_consolidation,658xx,MO,13.13,0.0,Oct-03,0.0,NaN,5.0,0.0,5864.0,47.70%,9.0,6717.950109,6717.95,6000.00,717.95,May-14,211.41,NaN,May-14,0
8,1016373,1243872,25600.0,25600.0,25472.82947,60 months,14.27,599.26,4 years,RENT,86000.0,Verified,Nov-11,Fully Paid,debt_consolidation,105xx,NY,26.51,0.0,Oct-83,1.0,NaN,16.0,0.0,33021.0,70.80%,32.0,32840.056740,32659.13,25600.00,7240.06,Apr-14,16083.78,NaN,Jun-16,0
9,786870,990345,19750.0,19750.0,19750.00000,60 months,23.22,559.27,10+ years,MORTGAGE,72500.0,Verified,Jun-11,Fully Paid,debt_consolidation,630xx,MO,19.96,0.0,Mar-92,0.0,61.0,15.0,0.0,21544.0,98.70%,44.0,27544.891160,27544.89,19750.00,7794.89,Jun-13,15264.34,NaN,Feb-16,0


In [117]:
# Cambiamos los nombres de las columnas a castellano para que sea mas entendible
df = df.rename(columns={
    "loan_amnt": "importe_prestamo",
    "funded_amnt": "importe_financiado",
    "funded_amnt_inv": "importe_inversores",
    "term": "plazo_meses",
    "int_rate": "tasa_interes",
    "installment": "cuota_mensual",
    "emp_length": "antiguedad_laboral",
    "home_ownership": "tipo_vivienda",
    "annual_inc": "ingreso_anual",
    "verification_status": "estado_verificacion",
    "issue_d": "fecha_emision",
    "purpose": "proposito",
    "addr_state": "estado",
    "dti": "ratio_endeudamiento",
    "delinq_2yrs": "moras_ultimos_2_anos",
    "earliest_cr_line": "fecha_primera_linea_credito",
    "inq_last_6mths": "consultas_6_meses",
    "mths_since_last_delinq": "meses_desde_ultima_mora",
    "open_acc": "cuentas_abiertas",
    "pub_rec": "registros_publicos",
    "revol_bal": "saldo_rotativo",
    "revol_util": "utilizacion_rotativo",
    "total_acc": "total_cuentas",
    "repay_fail": "impago"
})


In [118]:
# Vemos si se aplicaron los cambios
df.columns

Index(['importe_prestamo', 'importe_financiado', 'importe_inversores',
       'plazo_meses', 'tasa_interes', 'cuota_mensual', 'antiguedad_laboral',
       'tipo_vivienda', 'ingreso_anual', 'estado_verificacion',
       'fecha_emision', 'proposito', 'estado', 'ratio_endeudamiento',
       'moras_ultimos_2_anos', 'fecha_primera_linea_credito',
       'consultas_6_meses', 'meses_desde_ultima_mora', 'cuentas_abiertas',
       'registros_publicos', 'saldo_rotativo', 'utilizacion_rotativo',
       'total_cuentas', 'impago'],
      dtype='object')

In [119]:
# Vemos la forma del dataset
df.shape

(38480, 24)

In [120]:
# Como podemos ver existen demasiadas columnas, habrá que eliminar las que no aporten valor


Vemos los valores nulos


In [121]:
# Tabla de valores NaN: conteo y porcentaje por columna
nulos = pd.DataFrame({
    "Nulos": df.isna().sum(),
    "Proporción": df.isna().mean() 
}).sort_values("Proporción", ascending=False)

nulos


,Nulos,Proporción
meses_desde_ultima_mora,24363,0.633134
antiguedad_laboral,993,0.025806
utilizacion_rotativo,59,0.001533
saldo_rotativo,4,0.000104
ingreso_anual,2,0.000052
importe_prestamo,1,0.000026
importe_inversores,1,0.000026
total_cuentas,1,0.000026
cuota_mensual,1,0.000026
registros_publicos,1,0.000026


Definimos la columna objetivo y eliminamos las columnas que no aportan valor

In [123]:
y = df["impago"]   # 0 = no impago, 1 = impago


In [ ]:
# Eliminamos las columnas que le den sobreinformación al modelo (leakage) o que sean posteriores al momento de petición del préstamo

# id: Identificador interno del préstamo, no aporta información útil.
# id_cliente: Identificador del cliente; el modelo no debe aprender esta información
# estado_prestamo: Estado final del préstamo; revela el resultado real al modelo
# total_pagado: Total pagado al final; depende del comportamiento futuro del cliente.
# total_pagado_inversor: Total pagado al inversor; información posterior al préstamo.
# principal_recuperado: Principal recuperado; refleja pagos reales ya ocurridos.
# intereses_recuperados: Intereses recuperados; también es información posterior.
# fecha_ultimo_pago: Fecha del último pago; solo se conoce después del préstamo.
# monto_ultimo_pago: Monto del último pago; indica si pagó o no, el modelo no debe aprender esta información
# fecha_proximo_pago: Próxima fecha de pago; no está disponible en el momento de la solicitud.
# fecha_revision_credito: Última fecha de revisión del crédito; puede ocurrir tras la emisión del préstamo.
# codigo_postal: Código postal truncado, aporta poca información y añade ruido.


In [ ]:
drop_columns = [ "id","id_cliente","estado_prestamo","total_pagado","total_pagado_inversor",
                  "principal_recuperado","intereses_recuperados","fecha_ultimo_pago","monto_ultimo_pago",
                  "fecha_proximo_pago","fecha_revision_credito","codigo_postal"]

df = df.drop(columns=drop_columns)



In [111]:
# Reviso que se hayn eliminado correctamente
df.shape

(38480, 24)

In [125]:
# Una vez eliminadas las columnas no necesarias hay que limpiar las que tienen datos en formatos inválidos

# utilizacion_rotativo: Porcentaje de utilización del crédito rotativo; viene con el signo % y también con caracteres extraños,por lo que debe limpiarse antes de convertirlo a un valor numérico.
# antiguedad_laboral: Antigüedad laboral del solicitante; está escrita en texto (“< 1 year”, “10+ years”, “4 years”),por lo que hay que extraer el número y convertirlo a formato numérico.
# fecha_emision y fecha_primera_linea_credito: Fechas del tipo “Dec-99” o “Jan-01” que vienen como texto; deben convertirse a formato datetime para poder trabajar con ellas.
# tasa_interes: Tipo de interés del préstamo; aparece expresado como texto con el símbolo %, lo que impide usarlo como número hasta limpiar el carácter %.
# plazo_meses: Duración del préstamo (“36 months”, “60 months”); es texto y debe extraerse el valor numérico de los meses.
# meses_desde_ultima_mora: Meses desde la última morosidad; contiene valores escritos como texto en vez de números, por lo que hay que convertirlos a numérico.
# saldo_rotativo: "Balance revolvente"; puede aparecer como texto en algunos registros, por lo que debe convertirse a dato numérico después de limpiar el formato.

In [126]:
columnas_a_limpiarse = ["utilizacion_rotativo", "tasa_interes", "plazo_meses", "meses_desde_ultima_mora", "saldo_rotativo"]

def solo_numeros_y_punto(valor):
    valor = str(valor)
    permitido = ["0","1","2","3","4","5","6","7","8","9","."]
    limpio = ""
    for i in valor:
        if i in permitido:
            limpio += i
    return limpio


# Aplicar limpieza a las columnas
for col in columnas_a_limpiarse:
    df[col] = df[col].astype(str).apply(solo_numeros_y_punto)
    df[col] = pd.to_numeric(df[col], errors="coerce")


In [127]:
# Limpieza de fechas
df["fecha_emision"] = pd.to_datetime(df["fecha_emision"], format="%b-%y", errors="coerce")
df["fecha_primera_linea_credito"] = pd.to_datetime(df["fecha_primera_linea_credito"], format="%b-%y", errors="coerce")


Revisamos el dataframe

In [115]:
df.head(15)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,repay_fail
0,0.0,0.0,0.00000,36,0.00,0.00,< 1 year,RENT,0.0,Not Verified,1999-12-01,major_purchase,ZZ,0.00,0.0,2001-01-01,0.0,NaN,0.0,0.0,0.0,0.00,1.0,1
1,2500.0,2500.0,2500.00000,36,13.98,85.42,4 years,RENT,20004.0,Not Verified,2010-07-01,other,MI,19.86,0.0,2005-08-01,5.0,NaN,7.0,0.0,981.0,21.30,10.0,0
2,5000.0,5000.0,5000.00000,36,15.95,175.67,4 years,RENT,59000.0,Not Verified,2010-06-01,debt_consolidation,NY,19.57,0.0,1994-04-01,1.0,59.0,7.0,0.0,18773.0,99.90,15.0,1
3,7000.0,7000.0,7000.00000,36,9.91,225.58,10+ years,MORTGAGE,53796.0,Not Verified,2011-09-01,other,TX,10.80,3.0,1998-03-01,3.0,3.0,7.0,0.0,3269.0,47.20,20.0,0
4,2000.0,2000.0,2000.00000,36,5.42,60.32,10+ years,RENT,30000.0,Not Verified,2011-09-01,debt_consolidation,NY,3.60,0.0,1975-01-01,0.0,72.0,7.0,0.0,0.0,0.00,15.0,0
5,3600.0,3600.0,3600.00000,36,10.25,116.59,10+ years,MORTGAGE,675048.0,Not Verified,2010-04-01,other,AL,1.55,0.0,1998-04-01,4.0,25.0,8.0,0.0,0.0,0.00,25.0,0
6,8000.0,8000.0,8000.00000,36,6.03,243.49,NaN,MORTGAGE,77736.0,Verified,2011-10-01,other,AZ,6.07,0.0,1996-07-01,0.0,NaN,12.0,0.0,4182.0,13.60,49.0,0
7,6000.0,6000.0,6000.00000,36,7.49,186.61,3 years,RENT,35000.0,Not Verified,2011-05-01,debt_consolidation,MO,13.13,0.0,2003-10-01,0.0,NaN,5.0,0.0,5864.0,47.70,9.0,0
8,25600.0,25600.0,25472.82947,60,14.27,599.26,4 years,RENT,86000.0,Verified,2011-11-01,debt_consolidation,NY,26.51,0.0,1983-10-01,1.0,NaN,16.0,0.0,33021.0,70.80,32.0,0
9,19750.0,19750.0,19750.00000,60,23.22,559.27,10+ years,MORTGAGE,72500.0,Verified,2011-06-01,debt_consolidation,MO,19.96,0.0,1992-03-01,0.0,61.0,15.0,0.0,21544.0,98.70,44.0,0
